In [ ]:
# Challenge Foundation Setup
# Run this cell FIRST before any challenge code

import subprocess
import os
import time
import math
import random
from IPython.display import display, HTML

def run_ros2_command(cmd, cwd="/home/jovyan", background=False):
    """Execute ROS2 commands and display output in notebook - TESTED VERSION"""
    
    # Complete environment setup for ROS2 in container
    env = os.environ.copy()
    env.update({
        'ROS_DOMAIN_ID': os.environ.get('ROS_DOMAIN_ID', '0'),
        'RMW_IMPLEMENTATION': os.environ.get('RMW_IMPLEMENTATION', 'rmw_fastrtps_cpp'),
        'DISPLAY': ':1',
        'ROS_DISTRO': 'jazzy',
        'AMENT_PREFIX_PATH': '/opt/ros/jazzy',
        'LD_LIBRARY_PATH': '/opt/ros/jazzy/lib:/opt/ros/jazzy/lib/x86_64-linux-gnu',
        'PATH': '/opt/ros/jazzy/bin:' + env.get('PATH', ''),
        'PYTHONPATH': '/opt/ros/jazzy/lib/python3.12/site-packages:' + env.get('PYTHONPATH', ''),
        'CMAKE_PREFIX_PATH': '/opt/ros/jazzy',
        'PKG_CONFIG_PATH': '/opt/ros/jazzy/lib/pkgconfig',
        'COLCON_PREFIX_PATH': '/opt/ros/jazzy'
    })
    
    # Use bash explicitly and source setup files
    bash_cmd = f"""
    bash -c '
    source /opt/ros/jazzy/setup.bash 2>/dev/null || true
    export ROS_DOMAIN_ID={env["ROS_DOMAIN_ID"]}
    export RMW_IMPLEMENTATION={env["RMW_IMPLEMENTATION"]}
    {cmd}
    '
    """
    
    try:
        if background:
            # Run in background for continuous commands
            process = subprocess.Popen(bash_cmd, shell=True, cwd=cwd, env=env)
            print(f"Background command started: {cmd}")
            return process
        else:
            result = subprocess.run(
                bash_cmd,
                shell=True, capture_output=True, text=True, cwd=cwd, env=env, timeout=10
            )
            
            print(f"Command: {cmd}")
            if result.stdout.strip():
                print(f"Output:\n{result.stdout}")
            if result.stderr.strip() and not "QStandardPaths" in result.stderr:
                print(f"Errors:\n{result.stderr}")
            return result
    except subprocess.TimeoutExpired:
        print(f"Command timed out: {cmd}")
        return None
    except Exception as e:
        print(f"Command failed: {e}")
        return None

def test_ros2_connection():
    """Test if ROS2 is working properly - ESSENTIAL CHECK"""
    print("🔍 Testing ROS2 connection...")
    
    # Test 1: Check if ros2 command works
    result1 = run_ros2_command("ros2 --help")
    
    # Test 2: List topics
    print("\n--- Available Topics ---")
    result2 = run_ros2_command("ros2 topic list")
    
    # Test 3: Check if turtlesim is running
    print("\n--- Active Nodes ---")
    result3 = run_ros2_command("ros2 node list")
    
    # Quick analysis
    if result2 and "/turtle1/cmd_vel" in result2.stdout:
        print("✅ TurtleSim is running and ready!")
    else:
        print("⚠️  TurtleSim may not be running. Start it in the desktop terminal:")
        print("   ros2 run turtlesim turtlesim_node")
    
    return result1, result2, result3

def quick_turtle_test():
    """Quick test to move turtle and verify everything works"""
    print("🐢 Quick turtle movement test...")
    
    # Move forward briefly
    result = run_ros2_command('ros2 topic pub --once /turtle1/cmd_vel geometry_msgs/msg/Twist "{linear: {x: 1.0, y: 0.0, z: 0.0}, angular: {x: 0.0, y: 0.0, z: 0.0}}"')
    
    if result and result.returncode == 0:
        print("✅ Turtle movement command successful!")
        print("🎯 Check your desktop window - turtle should have moved!")
    else:
        print("❌ Turtle movement failed. Check if TurtleSim is running.")
    
    return result

def display_challenge_header(challenge_num, title, learning_objectives):
    """Display formatted challenge header"""
    header_html = f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 15px; margin: 20px 0;">
        <h1 style="margin: 0 0 15px 0;">🎯 Challenge {challenge_num}: {title}</h1>
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;">
            <h3 style="margin: 0 0 10px 0;">📚 Learning Objectives:</h3>
            <ul style="margin: 0; padding-left: 20px;">
    """
    
    for objective in learning_objectives:
        header_html += f"<li>{objective}</li>"
    
    header_html += """
            </ul>
        </div>
    </div>
    """
    
    display(HTML(header_html))

def mark_task_complete(task_name):
    """Mark a task as completed with visual feedback"""
    complete_html = f"""
    <div style="background: #d4edda; color: #155724; padding: 10px; border-radius: 8px; border-left: 4px solid #28a745; margin: 10px 0;">
        <strong>✅ COMPLETED:</strong> {task_name}
    </div>
    """
    display(HTML(complete_html))

def show_hint(hint_text):
    """Display a hint in a styled box"""
    hint_html = f"""
    <div style="background: #fff3cd; color: #856404; padding: 15px; border-radius: 8px; border-left: 4px solid #ffc107; margin: 10px 0;">
        <strong>💡 HINT:</strong> {hint_text}
    </div>
    """
    display(HTML(hint_html))

def show_debug_info():
    """Show debugging information for troubleshooting"""
    debug_html = """
    <div style="background: #f8d7da; color: #721c24; padding: 15px; border-radius: 8px; border-left: 4px solid #dc3545; margin: 10px 0;">
        <strong>🔧 DEBUG MODE:</strong> Use this when things aren't working
    </div>
    """
    display(HTML(debug_html))
    
    print("Environment variables:")
    print(f"ROS_DOMAIN_ID: {os.environ.get('ROS_DOMAIN_ID', 'Not set')}")
    print(f"RMW_IMPLEMENTATION: {os.environ.get('RMW_IMPLEMENTATION', 'Not set')}")
    print(f"DISPLAY: {os.environ.get('DISPLAY', 'Not set')}")
    
    print("\nROS2 system check:")
    run_ros2_command("ros2 doctor --report")

# Essential utility functions for challenges
def parse_pose_output(pose_text):
    """Parse turtle pose from ros2 topic echo output"""
    try:
        lines = pose_text.strip().split('\n')
        x, y, theta = None, None, None
        
        for line in lines:
            if 'x:' in line:
                x = float(line.split(':')[1].strip())
            elif 'y:' in line:
                y = float(line.split(':')[1].strip())  
            elif 'theta:' in line:
                theta = float(line.split(':')[1].strip())
        
        return {'x': x, 'y': y, 'theta': theta}
    except:
        return {'x': 0, 'y': 0, 'theta': 0}

def degrees_to_radians(degrees):
    """Convert degrees to radians"""
    return degrees * math.pi / 180

def radians_to_degrees(radians):
    """Convert radians to degrees"""
    return radians * 180 / math.pi

print("🚀 Foundation code loaded successfully!")
print("📋 Available functions:")
print("  • run_ros2_command(cmd) - Execute ROS2 commands")
print("  • test_ros2_connection() - Check ROS2 setup")  
print("  • quick_turtle_test() - Test turtle movement")
print("  • display_challenge_header() - Format challenge display")
print("  • mark_task_complete() - Mark tasks done")
print("  • show_hint() - Display helpful hints")
print("  • show_debug_info() - Troubleshooting info")
print("\n🔧 Next step: Run test_ros2_connection() to verify setup")

In [ ]:
# Simple command to clear
run_ros2_command("ros2 service call /clear std_srvs/srv/Empty")

In [ ]:
# ============================================================================
# CHALLENGE 1 SOLUTION: Position Detective
# ============================================================================

def get_turtle_position_complete():
    """Complete solution for getting turtle position"""
    result = run_ros2_command("ros2 topic echo /turtle1/pose --once")
    
    if not result or result.returncode != 0:
        return None
    
    # Parse the output
    pose_data = parse_pose_output(result.stdout)
    
    # Convert theta to degrees
    angle_degrees = radians_to_degrees(pose_data['theta'])
    
    print(f"Turtle Position:")
    print(f"  X: {pose_data['x']:.2f}")
    print(f"  Y: {pose_data['y']:.2f}")
    print(f"  Angle: {angle_degrees:.1f}°")
    
    return pose_data

def position_logger_complete():
    """Complete solution for position logging with movement"""
    print("Taking position snapshot before movement...")
    pos_before = get_turtle_position_complete()
    
    print("\nMoving turtle forward...")
    run_ros2_command('ros2 topic pub --once /turtle1/cmd_vel geometry_msgs/msg/Twist "{linear: {x: 2.0, y: 0.0, z: 0.0}, angular: {x: 0.0, y: 0.0, z: 0.0}}"')
    
    time.sleep(2)  # Wait for movement
    
    print("\nTaking position snapshot after movement...")
    pos_after = get_turtle_position_complete()
    
    if pos_before and pos_after:
        # Calculate distance traveled
        dx = pos_after['x'] - pos_before['x']
        dy = pos_after['y'] - pos_before['y']
        distance = math.sqrt(dx*dx + dy*dy)
        
        print(f"\nMovement Analysis:")
        print(f"  Distance traveled: {distance:.2f} units")
        print(f"  X change: {dx:.2f}")
        print(f"  Y change: {dy:.2f}")
        
        return {'before': pos_before, 'after': pos_after, 'distance': distance}

def boundary_checker_complete():
    """Complete solution for boundary checking"""
    pos = get_turtle_position_complete()
    
    if not pos:
        print("Could not get position")
        return
    
    warnings = []
    
    if pos['x'] < 1.0:
        warnings.append("Near LEFT boundary!")
    elif pos['x'] > 10.0:
        warnings.append("Near RIGHT boundary!")
    
    if pos['y'] < 1.0:
        warnings.append("Near BOTTOM boundary!")
    elif pos['y'] > 10.0:
        warnings.append("Near TOP boundary!")
    
    if warnings:
        print("⚠️ BOUNDARY WARNINGS:")
        for warning in warnings:
            print(f"  {warning}")
    else:
        print("✅ Turtle is in safe zone")


In [ ]:
def get_turtle_position_clean():
    """Clean version that handles the formatting better"""
    result = run_ros2_command("ros2 topic echo /turtle1/pose --once")
    
    if not result or result.returncode != 0:
        print("Failed to get turtle position")
        return None
    
    # Split output into lines and look for the data we need
    lines = result.stdout.split('\n')
    x, y, theta = None, None, None
    
    for line in lines:
        line = line.strip()
        if line.startswith('x:'):
            x = float(line.split(':')[1].strip())
        elif line.startswith('y:'):
            y = float(line.split(':')[1].strip())
        elif line.startswith('theta:'):
            theta = float(line.split(':')[1].strip())
    
    if x is not None and y is not None and theta is not None:
        # Convert theta to degrees
        theta_degrees = theta * 180 / 3.14159
        
        print(f"Turtle Position:")
        print(f"  X: {x:.2f}")
        print(f"  Y: {y:.2f}") 
        print(f"  Angle: {theta_degrees:.1f}°")
        
        return {'x': x, 'y': y, 'theta': theta}
    else:
        print("Could not parse turtle position data")
        return None

# Test the cleaner version
get_turtle_position_clean()

In [ ]:
def get_turtle_position_complete():
    """Complete solution for getting turtle position"""
    result = run_ros2_command("ros2 topic echo /turtle1/pose --once")
    
    if not result or result.returncode != 0:
        return None
    
    # Parse the output
    pose_data = parse_pose_output(result.stdout)
    
    # Convert theta to degrees
    angle_degrees = radians_to_degrees(pose_data['theta'])
    
    print(f"Turtle Position:")
    print(f"  X: {pose_data['x']:.2f}")
    print(f"  Y: {pose_data['y']:.2f}")
    print(f"  Angle: {angle_degrees:.1f}°")
    
    return pose_data

get_turtle_position_complete()

In [ ]:
# ============================================================================
# CHALLENGE 2 SOLUTION: Precision Movement Workshop
# ============================================================================

def move_distance_precise_complete(distance, speed=1.0):
    """Complete precise movement solution"""
    print(f"Moving {distance} units at speed {speed}")
    
    # Calculate movement time
    move_time = distance / speed
    
    # Start moving
    start_cmd = f'ros2 topic pub --once /turtle1/cmd_vel geometry_msgs/msg/Twist "{{linear: {{x: {speed}, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 0.0}}}}"'
    run_ros2_command(start_cmd)
    
    # Wait for precise time
    time.sleep(move_time)
    
    # Stop the turtle
    stop_cmd = 'ros2 topic pub --once /turtle1/cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.0, y: 0.0, z: 0.0}, angular: {x: 0.0, y: 0.0, z: 0.0}}"'
    run_ros2_command(stop_cmd)
    
    print("Movement complete!")

def rotate_degrees_precise_complete(degrees, angular_speed=1.0):
    """Complete precise rotation solution"""
    print(f"Rotating {degrees} degrees at angular speed {angular_speed}")
    
    # Convert degrees to radians
    target_radians = degrees_to_radians(degrees)
    
    # Calculate rotation time
    rotate_time = abs(target_radians) / angular_speed
    
    # Determine rotation direction
    angular_vel = angular_speed if degrees > 0 else -angular_speed
    
    # Start rotating
    start_cmd = f'ros2 topic pub --once /turtle1/cmd_vel geometry_msgs/msg/Twist "{{linear: {{x: 0.0, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: {angular_vel}}}}}"'
    run_ros2_command(start_cmd)
    
    # Wait for precise time
    time.sleep(rotate_time)
    
    # Stop rotation
    stop_cmd = 'ros2 topic pub --once /turtle1/cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.0, y: 0.0, z: 0.0}, angular: {x: 0.0, y: 0.0, z: 0.0}}"'
    run_ros2_command(stop_cmd)
    
    print("Rotation complete!")

def draw_perfect_square_complete(side_length):
    """Complete solution for drawing a perfect square"""
    print(f"Drawing perfect square with side length {side_length}")
    
    for i in range(4):
        print(f"Side {i+1}/4")
        
        # Move forward for one side
        move_distance_precise_complete(side_length, speed=1.0)
        
        # Wait a moment
        time.sleep(0.5)
        
        # Rotate 90 degrees
        rotate_degrees_precise_complete(90, angular_speed=1.0)
        
        # Wait a moment
        time.sleep(0.5)
    
    print("Perfect square completed!")


In [ ]:
# ============================================================================
# CHALLENGE 3 SOLUTION: Multi-Turtle Controller
# ============================================================================
def spawn_turtle_complete(name, x=5.0, y=5.0):
    """Complete turtle spawning solution"""
    cmd = f'ros2 service call /spawn turtlesim/srv/Spawn "{{x: {x}, y: {y}, theta: 0.0, name: {name}}}"'
    result = run_ros2_command(cmd)
    print(f"Spawned turtle '{name}' at ({x}, {y})")
    return result

def move_specific_turtle_complete(turtle_name, linear_x=1.0, angular_z=0.0):
    """Complete solution for moving specific turtle"""
    # Correct topic naming: /turtlename/cmd_vel (not /turtle + name)
    topic_name = f"/{turtle_name}/cmd_vel"
    
    cmd = f'ros2 topic pub --once {topic_name} geometry_msgs/msg/Twist "{{linear: {{x: {linear_x}, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: {angular_z}}}}}"'
    result = run_ros2_command(cmd)
    print(f"Moving {turtle_name}: linear={linear_x}, angular={angular_z}")
    return result

def get_active_turtles_complete():
    """Complete solution for getting active turtle list - FIXED PARSING"""
    result = run_ros2_command("ros2 node list")
    
    if not result or result.returncode != 0:
        return []
    
    turtles = []
    lines = result.stdout.strip().split('\n')
    
    for line in lines:
        line = line.strip()
        # Look for turtle nodes (they start with / and contain turtle)
        if line.startswith('/') and ('turtle' in line or line == '/turtlesim'):
            if line != '/turtlesim':  # Skip the turtlesim server node
                turtle_name = line[1:]  # Remove leading '/'
                if turtle_name not in turtles:  # Avoid duplicates
                    turtles.append(turtle_name)
    
    print(f"Active turtles found: {turtles}")
    return turtles

def formation_forward_complete(distance=2.0):
    """Complete formation movement solution"""
    turtles = get_active_turtles_complete()
    
    if not turtles:
        print("No turtles found!")
        return
    
    print(f"Moving {len(turtles)} turtles in formation...")
    
    # Move all turtles simultaneously
    for turtle in turtles:
        move_specific_turtle_complete(turtle, linear_x=1.0, angular_z=0.0)
    
    # Wait for movement
    time.sleep(distance)
    
    # Stop all turtles
    for turtle in turtles:
        topic_name = f"/{turtle}/cmd_vel"
        stop_cmd = f'ros2 topic pub --once {topic_name} geometry_msgs/msg/Twist "{{linear: {{x: 0.0, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 0.0}}}}"'
        run_ros2_command(stop_cmd)
    
    print("Formation movement complete!")

def follow_the_leader_complete():
    """Complete follow-the-leader solution"""
    turtles = get_active_turtles_complete()
    
    if len(turtles) < 2:
        print("Need at least 2 turtles for follow-the-leader")
        return
    
    leader = turtles[0]
    followers = turtles[1:]
    
    print(f"Leader: {leader}, Followers: {followers}")
    
    # Leader moves first
    print("Leader moving...")
    move_specific_turtle_complete(leader, linear_x=2.0, angular_z=0.0)
    
    # Followers follow with delay
    for i, follower in enumerate(followers):
        time.sleep(1)  # 1 second delay between followers
        print(f"Follower {follower} following...")
        move_specific_turtle_complete(follower, linear_x=2.0, angular_z=0.0)

spawn_turtle_complete("turtle2", x=6, y=6)
move_specific_turtle_complete("turtle2",1.0, 1.0)

In [ ]:
# ============================================================================
# CHALLENGE 3 SOLUTION: Multi-Turtle Controller
# ============================================================================

def spawn_turtle_complete(name, x=5.0, y=5.0):
    """Complete turtle spawning solution"""
    cmd = f'ros2 service call /spawn turtlesim/srv/Spawn "{{x: {x}, y: {y}, theta: 0.0, name: {name}}}"'
    result = run_ros2_command(cmd)
    print(f"Spawned turtle '{name}' at ({x}, {y})")
    return result

def move_specific_turtle_complete(turtle_name, linear_x=1.0, angular_z=0.0):
    """Complete solution for moving specific turtle"""
    # Correct topic naming: /turtlename/cmd_vel (not /turtle + name)
    topic_name = f"/{turtle_name}/cmd_vel"
    
    cmd = f'ros2 topic pub --once {topic_name} geometry_msgs/msg/Twist "{{linear: {{x: {linear_x}, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: {angular_z}}}}}"'
    result = run_ros2_command(cmd)
    print(f"Moving {turtle_name}: linear={linear_x}, angular={angular_z}")
    return result

def get_active_turtles_complete():
    """Complete solution for getting active turtle list - FIXED PARSING"""
    result = run_ros2_command("ros2 node list")
    
    if not result or result.returncode != 0:
        return []
    
    turtles = []
    lines = result.stdout.strip().split('\n')
    
    for line in lines:
        line = line.strip()
        # Look for turtle nodes (they start with / and contain turtle)
        if line.startswith('/') and ('turtle' in line or line == '/turtlesim'):
            if line != '/turtlesim':  # Skip the turtlesim server node
                turtle_name = line[1:]  # Remove leading '/'
                if turtle_name not in turtles:  # Avoid duplicates
                    turtles.append(turtle_name)
    
    print(f"Active turtles found: {turtles}")
    return turtles

def formation_forward_complete(distance=2.0):
    """Complete formation movement solution"""
    turtles = get_active_turtles_complete()
    
    if not turtles:
        print("No turtles found!")
        return
    
    print(f"Moving {len(turtles)} turtles in formation...")
    
    # Move all turtles simultaneously
    for turtle in turtles:
        move_specific_turtle_complete(turtle, linear_x=1.0, angular_z=0.0)
    
    # Wait for movement
    time.sleep(distance)
    
    # Stop all turtles
    for turtle in turtles:
        topic_name = f"/{turtle}/cmd_vel"
        stop_cmd = f'ros2 topic pub --once {topic_name} geometry_msgs/msg/Twist "{{linear: {{x: 0.0, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 0.0}}}}"'
        run_ros2_command(stop_cmd)
    
    print("Formation movement complete!")

def follow_the_leader_complete():
    """Complete follow-the-leader solution"""
    turtles = get_active_turtles_complete()
    
    if len(turtles) < 2:
        print("Need at least 2 turtles for follow-the-leader")
        return
    
    leader = turtles[0]
    followers = turtles[1:]
    
    print(f"Leader: {leader}, Followers: {followers}")
    
    # Leader moves first
    print("Leader moving...")
    move_specific_turtle_complete(leader, linear_x=2.0, angular_z=0.0)
    
    # Followers follow with delay
    for i, follower in enumerate(followers):
        time.sleep(1)  # 1 second delay between followers
        print(f"Follower {follower} following...")
        move_specific_turtle_complete(follower, linear_x=2.0, angular_z=0.0)

In [ ]:
# ============================================================================
# CHALLENGE 5 SOLUTION: Treasure Hunt Game
# ============================================================================

class TreasureHunter_Complete:
    def __init__(self, turtle_name="turtle1"):
        self.turtle_name = turtle_name
        self.current_pos = {"x": 5.5, "y": 5.5, "theta": 0}
        self.treasure_pos = {"x": 9.0, "y": 9.0}
        
    def update_position_complete(self):
        """Complete solution for position tracking - FIXED PARSING"""
        result = run_ros2_command(f"ros2 topic echo /{self.turtle_name}/pose --once")
        
        if result and result.returncode == 0:
            # Use the improved parsing function
            pose_data = parse_pose_output(result.stdout)
            if pose_data['x'] is not None:  # Check if parsing was successful
                self.current_pos = pose_data
                return True
        
        print(f"Failed to update position for {self.turtle_name}")
        return False
    
    def calculate_distance_to_treasure(self):
        """Calculate distance to treasure"""
        dx = self.treasure_pos["x"] - self.current_pos["x"]
        dy = self.treasure_pos["y"] - self.current_pos["y"]
        return math.sqrt(dx*dx + dy*dy)
    
    def calculate_angle_to_treasure_complete(self):
        """Complete solution for angle calculation"""
        dx = self.treasure_pos["x"] - self.current_pos["x"]
        dy = self.treasure_pos["y"] - self.current_pos["y"]
        target_angle = math.atan2(dy, dx)
        
        # Calculate difference between current angle and target angle
        current_angle = self.current_pos["theta"]
        angle_diff = target_angle - current_angle
        
        # Handle angle wraparound (normalize to -π to π)
        while angle_diff > math.pi:
            angle_diff -= 2 * math.pi
        while angle_diff < -math.pi:
            angle_diff += 2 * math.pi
        
        return angle_diff
    
    def move_toward_treasure_complete(self):
        """Complete treasure seeking solution"""
        self.update_position_complete()
        
        distance = self.calculate_distance_to_treasure()
        angle_diff = self.calculate_angle_to_treasure_complete()
        
        print(f"Distance: {distance:.2f}, Angle diff: {radians_to_degrees(angle_diff):.1f}°")
        
        # If we need to turn more than 10 degrees, rotate first
        if abs(angle_diff) > 0.17:  # 10 degrees in radians
            # Rotate toward treasure
            angular_vel = 1.0 if angle_diff > 0 else -1.0
            cmd = f'ros2 topic pub --once /{self.turtle_name}/cmd_vel geometry_msgs/msg/Twist "{{linear: {{x: 0.0, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: {angular_vel}}}}}"'
            run_ros2_command(cmd)
            print("Rotating toward treasure...")
        else:
            # Move forward toward treasure
            move_speed = min(1.0, distance)  # Slow down when close
            cmd = f'ros2 topic pub --once /{self.turtle_name}/cmd_vel geometry_msgs/msg/Twist "{{linear: {{x: {move_speed}, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 0.0}}}}}"'
            run_ros2_command(cmd)
            print("Moving toward treasure...")
        
        return distance < 0.5  # Return True if treasure found
    
    def treasure_hunt_game_complete(self):
        """Complete treasure hunt game loop"""
        print("🏴‍☠️ Starting treasure hunt!")
        print(f"Treasure location: ({self.treasure_pos['x']}, {self.treasure_pos['y']})")
        
        moves = 0
        max_moves = 100
        
        while moves < max_moves:
            found = self.move_toward_treasure_complete()
            
            if found:
                print("🎉 TREASURE FOUND! Congratulations!")
                # Stop the turtle
                stop_cmd = f'ros2 topic pub --once /{self.turtle_name}/cmd_vel geometry_msgs/msg/Twist "{{linear: {{x: 0.0, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 0.0}}}}"'
                run_ros2_command(stop_cmd)
                break
            
            moves += 1
            time.sleep(0.5)
        
        if moves >= max_moves:
            print("❌ Treasure hunt failed - too many moves")
        else:
            print(f"✅ Treasure found in {moves} moves!")

In [ ]:
# ============================================================================
# CHALLENGE 6 SOLUTION: Turtle Communication Network
# ============================================================================
class TurtleMessenger_Complete:
    def __init__(self, my_name="turtle1"):
        self.my_name = my_name
        self.message_log = []
        
    def send_message_complete(self, recipient, message):
        """Complete messaging solution using ROS parameters"""
        timestamp = time.time()
        
        # Create message format
        message_data = {
            "sender": self.my_name,
            "recipient": recipient,
            "content": message,
            "timestamp": timestamp
        }
        
        # Store message as ROS parameter
        param_name = f"message_{recipient}_{int(timestamp)}"
        param_value = f"{self.my_name}|{message}|{timestamp}"
        
        run_ros2_command(f'ros2 param set /turtlesim {param_name} "{param_value}"')
        print(f"📤 {self.my_name} → {recipient}: {message}")
        
        return message_data
    
    def check_messages_complete(self):
        """Complete message checking solution - FIXED PARSING"""
        result = run_ros2_command("ros2 param list /turtlesim")
        
        if not result or result.returncode != 0:
            return []
        
        messages = []
        my_message_prefix = f"message_{self.my_name}"
        
        lines = result.stdout.strip().split('\n')
        param_names = []
        
        for line in lines:
            line = line.strip()
            # Skip command output lines
            if line and not line.startswith('CompletedProcess') and not line.startswith('bash'):
                param_names.append(line)
        
        for param_name in param_names:
            if param_name.startswith(my_message_prefix):
                # Get the message content
                msg_result = run_ros2_command(f"ros2 param get /turtlesim {param_name}")
                if msg_result and msg_result.returncode == 0:
                    # Parse message: "sender|content|timestamp"
                    try:
                        # Extract just the value part (after "String value is: ")
                        msg_lines = msg_result.stdout.strip().split('\n')
                        msg_content = None
                        for msg_line in msg_lines:
                            if 'String value is:' in msg_line:
                                msg_content = msg_line.split('String value is:')[1].strip()
                                break
                        
                        if msg_content:
                            sender, content, timestamp = msg_content.split('|')
                            messages.append({
                                "sender": sender,
                                "content": content,
                                "timestamp": float(timestamp)
                            })
                            # Delete the message after reading
                            run_ros2_command(f"ros2 param delete /turtlesim {param_name}")
                    except:
                        pass
        
        if messages:
            print(f"📥 {len(messages)} new messages for {self.my_name}")
            for msg in messages:
                print(f"  From {msg['sender']}: {msg['content']}")
        
        return messages
    
    def broadcast_location_complete(self):
        """Complete location broadcasting solution"""
        # Get current position
        result = run_ros2_command(f"ros2 topic echo /{self.my_name}/pose --once")
        
        if result and result.returncode == 0:
            pose_data = parse_pose_output(result.stdout)
            
            # Broadcast to all other turtles
            turtles = get_active_turtles_complete()
            location_msg = f"Location: ({pose_data['x']:.1f}, {pose_data['y']:.1f})"
            
            for turtle in turtles:
                if turtle != self.my_name:
                    self.send_message_complete(turtle, location_msg)
            
            print(f"📡 Broadcasted location to {len(turtles)-1} turtles")


In [31]:
# ============================================================================
# CHALLENGE 7 SOLUTION: Mini Robot Olympics
# ============================================================================
class OlympicsJudge_Complete:
    def __init__(self):
        self.scores = {}
        self.events = ["speed_race", "precision_drawing", "synchronized_swim"]
        
    def time_trial_complete(self, turtle_name, start_pos, end_pos):
        """Complete timing solution"""
        print(f"⏱️ Starting time trial for {turtle_name}")
        
        # Move turtle to start position first (simplified)
        run_ros2_command(f"ros2 service call /teleport_absolute turtlesim/srv/TeleportAbsolute '{{x: {start_pos[0]}, y: {start_pos[1]}, theta: 0.0}}'")
        time.sleep(1)
        
        # Calculate distance to target
        dx = end_pos[0] - start_pos[0]
        dy = end_pos[1] - start_pos[1]
        target_distance = math.sqrt(dx*dx + dy*dy)
        
        start_time = time.time()
        
        # Simple movement toward target
        topic_name = f"/{turtle_name}/cmd_vel"
        cmd = f'ros2 topic pub --once {topic_name} geometry_msgs/msg/Twist "{{linear: {{x: 2.0, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 0.0}}}}"'
        run_ros2_command(cmd)
        
        # Wait for movement (simplified - in reality would track actual position)
        time.sleep(target_distance / 2.0)
        
        # Stop turtle
        stop_cmd = f'ros2 topic pub --once {topic_name} geometry_msgs/msg/Twist "{{linear: {{x: 0.0, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 0.0}}}}"'
        run_ros2_command(stop_cmd)
        
        end_time = time.time()
        elapsed = end_time - start_time
        
        print(f"🏁 {turtle_name} finished in {elapsed:.2f} seconds")
        return elapsed
    
    def precision_test_complete(self, turtle_name, target_shape="square"):
        """Complete precision testing solution"""
        print(f"📐 Precision test: {turtle_name} drawing {target_shape}")
        
        # Record starting position
        result = run_ros2_command(f"ros2 topic echo /{turtle_name}/pose --once")
        if result and result.returncode == 0:
            start_pos = parse_pose_output(result.stdout)
        else:
            start_pos = {"x": 5.5, "y": 5.5, "theta": 0}
        
        # Draw the shape (using our previous square function as example)
        if target_shape == "square":
            for i in range(4):
                # Move forward
                move_specific_turtle_complete(turtle_name, linear_x=1.0, angular_z=0.0)
                time.sleep(2)
                
                # Rotate 90 degrees
                rotate_cmd = f'ros2 topic pub --once /{turtle_name}/cmd_vel geometry_msgs/msg/Twist "{{linear: {{x: 0.0, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 1.57}}}}"'
                run_ros2_command(rotate_cmd)
                time.sleep(1)
        
        # Check final position vs start position (simplified scoring)
        final_result = run_ros2_command(f"ros2 topic echo /{turtle_name}/pose --once")
        if final_result and final_result.returncode == 0:
            end_pos = parse_pose_output(final_result.stdout)
            
            # Calculate precision score based on return accuracy
            dx = abs(end_pos['x'] - start_pos['x'])
            dy = abs(end_pos['y'] - start_pos['y'])
            error = math.sqrt(dx*dx + dy*dy)
            
            # Score: 100 points minus error penalty
            score = max(0, 100 - (error * 20))
            print(f"🎯 Precision score: {score:.1f}/100 (error: {error:.2f})")
            return score
        
        return 50  # Default score if position tracking failed
    
    def calculate_speed_score(self, time_taken, target_time=10.0):
        """Convert time to score (0-100)"""
        if time_taken <= target_time:
            return 100
        else:
            penalty = (time_taken - target_time) * 5
            return max(0, 100 - penalty)
    
    def run_full_olympics_complete(self):
        """Complete olympics competition"""
        participants = get_active_turtles_complete()
        
        if len(participants) < 2:
            print("Need at least 2 turtles for olympics!")
            return
        
        print(f"🏆 ROBOT OLYMPICS STARTING!")
        print(f"Participants: {participants}")
        
        # Initialize scores
        for turtle in participants:
            self.scores[turtle] = 0
        
        # Event 1: Speed Race
        print(f"\n=== SPEED RACE EVENT ===")
        for turtle in participants:
            time_taken = self.time_trial_complete(turtle, [2, 2], [9, 9])
            score = self.calculate_speed_score(time_taken, target_time=5.0)
            self.scores[turtle] += score
            print(f"{turtle}: {score:.1f} points")
            time.sleep(2)  # Break between participants
        
        # Event 2: Precision Drawing
        print(f"\n=== PRECISION DRAWING EVENT ===")
        for turtle in participants:
            score = self.precision_test_complete(turtle, "square")
            self.scores[turtle] += score
            print(f"{turtle}: {score:.1f} points")
            time.sleep(2)
        
        # Event 3: Synchronized Swimming (simplified)
        print(f"\n=== SYNCHRONIZED SWIMMING EVENT ===")
        print("All turtles perform synchronized movement...")
        
        # Move all turtles in formation
        for turtle in participants:
            move_specific_turtle_complete(turtle, linear_x=1.0, angular_z=0.5)
        
        time.sleep(5)  # Let them dance
        
        # Stop all
        for turtle in participants:
            topic_name = f"/{turtle}/cmd_vel"
            run_ros2_command(f'ros2 topic pub --once {topic_name} geometry_msgs/msg/Twist "{{linear: {{x: 0.0, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 0.0}}}}"')
        
        # Award points for participation (in real scenario, would judge coordination)
        sync_points = 75
        for turtle in participants:
            self.scores[turtle] += sync_points
            print(f"{turtle}: {sync_points} points")
        
        # Final Results
        print(f"\n🏆 OLYMPICS FINAL RESULTS:")
        sorted_scores = sorted(self.scores.items(), key=lambda x: x[1], reverse=True)
        
        for i, (turtle, score) in enumerate(sorted_scores):
            medal = ["🥇", "🥈", "🥉"][i] if i < 3 else "🏅"
            print(f"{medal} {turtle}: {score:.1f} points")
        
        winner = sorted_scores[0][0]
        print(f"\n🎉 OLYMPICS CHAMPION: {winner}!")
        
        return sorted_scores

# ============================================================================
# TESTING AND DEMONSTRATION FUNCTIONS
# ============================================================================

def run_all_challenge_demos():
    """Demonstrate all challenge solutions"""
    print("🚀 Running all challenge demonstrations...")
    
    # Challenge 1 Demo
    print("\n" + "="*50)
    print("CHALLENGE 1 DEMO: Position Detective")
    print("="*50)
    get_turtle_position_complete()
    position_logger_complete()
    boundary_checker_complete()
    
    time.sleep(3)
    
    # Challenge 2 Demo  
    print("\n" + "="*50)
    print("CHALLENGE 2 DEMO: Precision Movement")
    print("="*50)
    move_distance_precise_complete(2.0, speed=1.0)
    time.sleep(2)
    rotate_degrees_precise_complete(90, angular_speed=1.0)
    time.sleep(2)
    draw_perfect_square_complete(2.0)
    
    time.sleep(3)
    
    # Challenge 3 Demo
    print("\n" + "="*50)
    print("CHALLENGE 3 DEMO: Multi-Turtle Controller")
    print("="*50)
    spawn_turtle_complete("alice", 3.0, 3.0)
    spawn_turtle_complete("bob", 8.0, 8.0)
    time.sleep(2)
    formation_forward_complete(2.0)
    time.sleep(3)
    follow_the_leader_complete()
    
    time.sleep(3)
    
    # Challenge 4 Demo
    print("\n" + "="*50)
    print("CHALLENGE 4 DEMO: Traffic Controller")
    print("="*50)
    controller = TrafficController_Complete()
    controller.get_turtle_parameters_complete()
    controller.change_turtle_color_complete(255, 0, 0)  # Red
    time.sleep(2)
    controller.stop_all_traffic_complete()
    time.sleep(2)
    controller.change_turtle_color_complete(0, 255, 0)  # Green
    
    time.sleep(3)
    
    # Challenge 5 Demo
    print("\n" + "="*50)
    print("CHALLENGE 5 DEMO: Treasure Hunter")
    print("="*50)
    hunter = TreasureHunter_Complete("turtle1")
    hunter.treasure_pos = {"x": 8.0, "y": 8.0}  # Set nearby treasure
    
    # Run a few treasure hunting steps
    for i in range(5):
        found = hunter.move_toward_treasure_complete()
        if found:
            break
        time.sleep(1)
    
    time.sleep(3)
    
    # Challenge 6 Demo
    print("\n" + "="*50)
    print("CHALLENGE 6 DEMO: Turtle Communication")
    print("="*50)
    messenger1 = TurtleMessenger_Complete("turtle1")
    messenger2 = TurtleMessenger_Complete("alice")
    
    messenger1.send_message_complete("alice", "Hello Alice!")
    messenger1.broadcast_location_complete()
    time.sleep(2)
    messenger2.check_messages_complete()
    
    time.sleep(3)
    
    # Challenge 7 Demo
    print("\n" + "="*50)
    print("CHALLENGE 7 DEMO: Robot Olympics")
    print("="*50)
    judge = OlympicsJudge_Complete()
    
    # Run mini olympics (shortened version for demo)
    participants = get_active_turtles_complete()
    if len(participants) >= 2:
        judge.run_full_olympics_complete()
    else:
        print("Need at least 2 turtles for Olympics demo")
    
    print("\n" + "="*50)
    print("ALL DEMONSTRATIONS COMPLETE!")
    print("="*50)


In [ ]:
# ============================================================================
# INSTRUCTOR UTILITIES
# ============================================================================

def reset_simulation():
    """Reset the simulation to clean state"""
    print("Resetting simulation...")
    
    # Clear the screen
    run_ros2_command("ros2 service call /clear std_srvs/srv/Empty")
    
    # Reset turtle1 to center
    run_ros2_command("ros2 service call /reset std_srvs/srv/Empty")
    
    # Reset background to default
    controller = TrafficController_Complete()
    controller.change_turtle_color_complete(69, 86, 255)  # Default blue
    
    print("Simulation reset complete!")

def create_challenge_test_environment():
    """Set up environment for testing all challenges"""
    print("Setting up challenge test environment...")
    
    # Reset first
    reset_simulation()
    time.sleep(2)
    
    # Spawn test turtles
    spawn_turtle_complete("alice", 3.0, 7.0)
    spawn_turtle_complete("bob", 7.0, 3.0)
    spawn_turtle_complete("charlie", 8.0, 8.0)
    
    print("Test environment ready with 4 turtles!")
    return ["turtle1", "alice", "bob", "charlie"]

def validate_student_solution(challenge_num, student_function, expected_behavior):
    """Validate student solutions against expected behavior"""
    print(f"Validating Challenge {challenge_num} solution...")
    
    try:
        # Test the student's function
        result = student_function()
        
        # Check if it meets expected behavior (simplified validation)
        if expected_behavior(result):
            print(f"✅ Challenge {challenge_num}: PASSED")
            return True
        else:
            print(f"❌ Challenge {challenge_num}: FAILED - Does not meet requirements")
            return False
            
    except Exception as e:
        print(f"❌ Challenge {challenge_num}: ERROR - {str(e)}")
        return False

In [ ]:
# ============================================================================
# STUDENT PROGRESS TRACKING
# ============================================================================

class StudentProgress:
    def __init__(self, student_name):
        self.student_name = student_name
        self.completed_challenges = {}
        self.start_time = time.time()
        
    def mark_challenge_complete(self, challenge_num, score=100):
        """Mark a challenge as completed"""
        self.completed_challenges[challenge_num] = {
            'score': score,
            'completion_time': time.time(),
            'elapsed': time.time() - self.start_time
        }
        
        print(f"Student {self.student_name}: Challenge {challenge_num} completed with score {score}")
        
    def get_progress_report(self):
        """Generate progress report"""
        total_challenges = 7
        completed = len(self.completed_challenges)
        avg_score = sum(c['score'] for c in self.completed_challenges.values()) / max(1, completed)
        
        report = f"""
        PROGRESS REPORT: {self.student_name}
        ================================
        Challenges Completed: {completed}/{total_challenges}
        Average Score: {avg_score:.1f}%
        Total Time: {(time.time() - self.start_time)/3600:.1f} hours
        
        Challenge Details:
        """
        
        for i in range(1, total_challenges + 1):
            if i in self.completed_challenges:
                c = self.completed_challenges[i]
                report += f"  Challenge {i}: ✅ Score: {c['score']:.1f}% (Time: {c['elapsed']/60:.1f}min)\n"
            else:
                report += f"  Challenge {i}: ⏳ Not completed\n"
        
        return report

# ============================================================================
# QUICK START FUNCTIONS FOR INSTRUCTORS
# ============================================================================

def instructor_quick_test():
    """Quick test of all major functions for instructors"""
    print("INSTRUCTOR QUICK TEST")
    print("="*40)
    
    # Test basic connectivity
    print("1. Testing ROS2 connection...")
    test_ros2_connection()
    
    print("\n2. Testing turtle movement...")
    quick_turtle_test()
    
    print("\n3. Testing position tracking...")
    get_turtle_position_complete()
    
    print("\n4. Testing multi-turtle spawning...")
    spawn_turtle_complete("test_turtle", 6.0, 6.0)
    
    print("\n5. Testing parameter modification...")
    controller = TrafficController_Complete()
    controller.change_turtle_color_complete(0, 255, 0)
    
    print("\n6. Cleaning up...")
    reset_simulation()
    
    print("\nQuick test complete! All systems working.")

def create_student_notebook_template(challenge_num):
    """Generate notebook template for specific challenge"""
    
    templates = {
        1: """# Challenge 1: Position Detective
# Copy the foundation code here first!

# Your task: Complete the get_turtle_position function
def get_turtle_position():
    '''Get current turtle position - STARTER VERSION'''
    result = run_ros2_command("ros2 topic echo /turtle1/pose --once")
    
    # TODO: Parse the result to extract x, y, and theta values
    # TODO: Convert theta from radians to degrees
    # TODO: Display in a user-friendly format
    
    return result

# Test your function
print("Testing position function:")
get_turtle_position()""",
        
        2: """# Challenge 2: Precision Movement Workshop
# Copy the foundation code here first!

def move_distance_precise(distance, speed=1.0):
    '''Move turtle exactly the specified distance - INCOMPLETE'''
    print(f"Moving {distance} units at speed {speed}")
    
    # Step 1: Start moving
    cmd_start = f'ros2 topic pub --once /turtle1/cmd_vel geometry_msgs/msg/Twist "{{linear: {{x: {speed}, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: 0.0}}}}"'
    run_ros2_command(cmd_start)
    
    # Step 2: Calculate how long to move
    move_time = distance / speed
    time.sleep(move_time)
    
    # Step 3: TODO - Add stop command here
    
    print("Movement complete!")

# Test with a small distance
move_distance_precise(1.0, speed=0.5)""",
        
        3: """# Challenge 3: Multi-Turtle Controller
# Copy the foundation code here first!

def spawn_turtle(name, x=5.0, y=5.0):
    '''Spawn a new turtle - STARTER VERSION'''
    cmd = f'ros2 service call /spawn turtlesim/srv/Spawn "{{x: {x}, y: {y}, theta: 0.0, name: {name}}}"'
    result = run_ros2_command(cmd)
    print(f"Spawning turtle '{name}' at ({x}, {y})")
    return result

def move_specific_turtle(turtle_name, linear_x=1.0, angular_z=0.0):
    '''Move a specific turtle - INCOMPLETE'''
    # TODO: Build the correct topic name for the specific turtle
    topic_name = f"/{turtle_name}/cmd_vel"  # Check if this is correct
    
    cmd = f'ros2 topic pub --once {topic_name} geometry_msgs/msg/Twist "{{linear: {{x: {linear_x}, y: 0.0, z: 0.0}}, angular: {{x: 0.0, y: 0.0, z: {angular_z}}}}}"'
    return run_ros2_command(cmd)

# Test spawning
spawn_turtle("alice", 2.0, 2.0)
move_specific_turtle("alice", 1.0, 0.0)"""
    }
    
    return templates.get(challenge_num, "# Template not found for this challenge number")

# Final setup
print("🎓 COMPLETE INSTRUCTOR SOLUTIONS LOADED")
print("📚 Available functions:")
print("  • All challenge solutions (*_complete functions)")
print("  • run_all_challenge_demos() - Demo all challenges")
print("  • instructor_quick_test() - Quick system test")
print("  • create_challenge_test_environment() - Setup test environment")
print("  • reset_simulation() - Clean reset")
print("  • create_student_notebook_template(n) - Generate student templates")
print("\n⚠️  REMINDER: These are instructor reference solutions!")
print("📝 Students should work through challenges step-by-step with guided templates")